In [1]:
import requests
import json
import pandas as pd
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import os
from getpass import getpass
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from dotenv import load_dotenv
import time
from pymongo import MongoClient
from dotenv import dotenv_values
import googlemaps

##### **I am going to look at other stores of the same franchise and take the top 20 in visits from Bogota, Medellin and Cali, then I will take the 4 best of each city and find out what they have in common.**

In [2]:
load_dotenv()

True

In [3]:
api_key = os.getenv("google_key") 

In [4]:
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
bogota = "Interrapidisimo Bogotá"

In [5]:

params = {
    "query": bogota,
    "key": api_key,
    "region": "co",
    "language": "es",
    "maxResults": 20
}



In [6]:
response = requests.get(url, params=params)
data = json.loads(response.text)

In [27]:
results = []

# Loop through the data and extract the necessary information
for result in data["results"]:
    name = result["name"]
    lat = result["geometry"]["location"]["lat"]
    lng = result["geometry"]["location"]["lng"]
    num_reviews = result.get("user_ratings_total", 0)
    address = result.get("formatted_address", "")
    results.append({
        "Name": name,
        "Latitude": lat,
        "Longitude": lng,
        "Number of Reviews": num_reviews,
        "Address": address
    })


In [28]:
df_bogota = pd.DataFrame(results)

In [29]:
df_bogota.sort_values(by="Number of Reviews", ascending=False)

,Name,Latitude,Longitude,Number of Reviews,Address
0,INTER RAPIDÍSIMO - Bogotá Cr. 30 # 7 -45,4.608304,-74.096978,2134,"Ave Cra 30 #7-45, Bogotá"
13,INTERRAPIDISIMO ORQUIDEAS,4.740178,-74.039899,172,"Cl. 161 #16B-48, Bogotá"
17,Inter Rapidisimo Cedritos,4.721523,-74.042458,161,"Cra. 16 #136-91, Bogotá"
10,inter rapidisimo,4.641318,-74.139985,90,"Cl. 9 #77-31, Bogotá"
12,Interrapidisimo Pio XII,4.634157,-74.148891,88,"Cra. 79 #6a-39, Kennedy, Bogotá, Cundinamarca"
3,INTER RAPIDISIMO CENTENARIO,4.588777,-74.109237,87,"Cra 27 #22-42, Bogotá"
14,INTERRAPIDISIMO SAN CRISTOBAL NORTE BOGOTÁ,4.739351,-74.028398,84,"110131, Bogotá, Cundinamarca"
1,INTER RAPIDÍSIMO BOGOTÁ,4.635360,-74.161764,83,"CRA 86 # 34 B -14 SUR, Bogotá"
6,Interrapidisimo,4.727663,-74.032663,73,"Cra. 10 # 149-23, Usaquén, Bogotá, Cundinamarca"
11,INTER RAPIDISIMO ISLA DEL SOL - MADELENA - TUNAL,4.588934,-74.142933,43,"DgG, Dg. 52 Sur #54 A 40, Bogotá"


In [10]:
import requests

# Define la URL de la API Geocoding de Google Maps
geocoding_url = "https://maps.googleapis.com/maps/api/geocode/json"


# Define una función para obtener el Place ID de una ubicación a partir de sus coordenadas de latitud y longitud
def get_place_id(lat, lng):
    # Define los parámetros de la solicitud de geocodificación inversa
    params = {
        "latlng": f"{lat},{lng}",
        "key": api_key,
    }
    # Realiza la solicitud a la API Geocoding de Google Maps
    response = requests.get(geocoding_url, params=params)
    # Convierte la respuesta a un diccionario de Python
    response_dict = response.json()
    # Extrae el Place ID de la respuesta
    place_id = response_dict["results"][0]["place_id"]
    # Retorna el Place ID
    return place_id

# Añade una nueva columna al dataframe con el Place ID de cada ubicación
df_bogota["Place ID"] = df_bogota.apply(lambda row: get_place_id(row["Latitude"], row["Longitude"]), axis=1)


In [30]:
df_bogota = df_bogota.sort_values(by="Number of Reviews", ascending=False)
df_bogota

,Name,Latitude,Longitude,Number of Reviews,Address
0,INTER RAPIDÍSIMO - Bogotá Cr. 30 # 7 -45,4.608304,-74.096978,2134,"Ave Cra 30 #7-45, Bogotá"
13,INTERRAPIDISIMO ORQUIDEAS,4.740178,-74.039899,172,"Cl. 161 #16B-48, Bogotá"
17,Inter Rapidisimo Cedritos,4.721523,-74.042458,161,"Cra. 16 #136-91, Bogotá"
10,inter rapidisimo,4.641318,-74.139985,90,"Cl. 9 #77-31, Bogotá"
12,Interrapidisimo Pio XII,4.634157,-74.148891,88,"Cra. 79 #6a-39, Kennedy, Bogotá, Cundinamarca"
3,INTER RAPIDISIMO CENTENARIO,4.588777,-74.109237,87,"Cra 27 #22-42, Bogotá"
14,INTERRAPIDISIMO SAN CRISTOBAL NORTE BOGOTÁ,4.739351,-74.028398,84,"110131, Bogotá, Cundinamarca"
1,INTER RAPIDÍSIMO BOGOTÁ,4.635360,-74.161764,83,"CRA 86 # 34 B -14 SUR, Bogotá"
6,Interrapidisimo,4.727663,-74.032663,73,"Cra. 10 # 149-23, Usaquén, Bogotá, Cundinamarca"
11,INTER RAPIDISIMO ISLA DEL SOL - MADELENA - TUNAL,4.588934,-74.142933,43,"DgG, Dg. 52 Sur #54 A 40, Bogotá"


In [31]:
# Crea un mapa centrado en Bogotá
map_bogota = folium.Map(location=[4.60971, -74.08175], zoom_start=12)

# Itera sobre cada fila de tu dataframe
for index, row in df_bogota.iterrows():
    # Obtén la latitud y longitud de la ubicación actual
    lat = row['Latitude']
    lng = row['Longitude']
    # Agrega un marcador en el mapa para la ubicación actual
    popup_text = f"{row['Name']}<br>Number of Reviews: {row['Number of Reviews']}"
    if row['Number of Reviews'] >= 80:
        marker_color = 'red'
    else:
        marker_color = 'blue'
    folium.Marker(location=[lat, lng], popup=popup_text, icon=folium.Icon(color=marker_color)).add_to(map_bogota)

# Visualiza el mapa
map_bogota

In [33]:
df_bogota["Address"]

0                              Ave Cra 30 #7-45, Bogotá
13                              Cl. 161 #16B-48, Bogotá
17                              Cra. 16 #136-91, Bogotá
10                                 Cl. 9 #77-31, Bogotá
12        Cra. 79 #6a-39, Kennedy, Bogotá, Cundinamarca
3                                 Cra 27 #22-42, Bogotá
14                         110131, Bogotá, Cundinamarca
1                         CRA 86 # 34 B -14 SUR, Bogotá
6       Cra. 10 # 149-23, Usaquén, Bogotá, Cundinamarca
11                     DgG, Dg. 52 Sur #54 A 40, Bogotá
15                               Cl. 150 #48-11, Bogotá
18                            Cra. 81a #13 D 39, Bogotá
19                              Cra. 82 #23A 25, Bogotá
7     Cra. 13 #57-28, Localidad de Chapinero, Bogotá...
2        Av. 1 de Mayo #51F - 47, Puente Aranda, Bogotá
9                 Cra. 28a #17-14, Los Mártires, Bogotá
5                               Cl. 79 #42 # 18, Bogotá
16       Cl. 72 #100 22, Engativá, Bogotá, Cundi

In [38]:
estratos = [3, 2, 4, 3, 2, 2, 3, 2, 5, 2, 5, 4, 4, 5, 3, 3, 5, 3, 3, 4]
df_bogota["Estratos"] = estratos


In [39]:
df_bogota

,Name,Latitude,Longitude,Number of Reviews,Address,Estratos
0,INTER RAPIDÍSIMO - Bogotá Cr. 30 # 7 -45,4.608304,-74.096978,2134,"Ave Cra 30 #7-45, Bogotá",3
13,INTERRAPIDISIMO ORQUIDEAS,4.740178,-74.039899,172,"Cl. 161 #16B-48, Bogotá",2
17,Inter Rapidisimo Cedritos,4.721523,-74.042458,161,"Cra. 16 #136-91, Bogotá",4
10,inter rapidisimo,4.641318,-74.139985,90,"Cl. 9 #77-31, Bogotá",3
12,Interrapidisimo Pio XII,4.634157,-74.148891,88,"Cra. 79 #6a-39, Kennedy, Bogotá, Cundinamarca",2
3,INTER RAPIDISIMO CENTENARIO,4.588777,-74.109237,87,"Cra 27 #22-42, Bogotá",2
14,INTERRAPIDISIMO SAN CRISTOBAL NORTE BOGOTÁ,4.739351,-74.028398,84,"110131, Bogotá, Cundinamarca",3
1,INTER RAPIDÍSIMO BOGOTÁ,4.635360,-74.161764,83,"CRA 86 # 34 B -14 SUR, Bogotá",2
6,Interrapidisimo,4.727663,-74.032663,73,"Cra. 10 # 149-23, Usaquén, Bogotá, Cundinamarca",5
11,INTER RAPIDISIMO ISLA DEL SOL - MADELENA - TUNAL,4.588934,-74.142933,43,"DgG, Dg. 52 Sur #54 A 40, Bogotá",2
